# Measure the computing performance (time, memory) of the model of septoria

In [1]:
# General useful imports
import random as rd
rd.seed(0)
import numpy as np
np.random.seed(0)
import pandas as pd

# Imports for wheat
from alinea.alep.wheat import adel_sample
from alinea.alep.architecture import set_properties, update_healthy_area, get_leaves
from alinea.adel.mtg_interpreter import *
from openalea.plantgl.all import *

# Imports for disease
from alinea.alep.protocol import *
from alinea.alep.septoria import plugin_septoria
from alinea.alep.disease_operation import generate_stock_lesions
from alinea.alep.dispersal_emission import BenchSeptoriaRainEmission
from alinea.alep.dispersal_transport import SeptoriaRainDispersal
from alinea.septo3d.alep_interfaces import Septo3DSplash
from alinea.alep.washing import RapillyWashing
from alinea.alep.growth_control import NoPriorityGrowthControl
from alinea.alep.infection_control import BiotrophDUPositionModel
from alinea.alep.senescence import WheatSeptoriaPositionedSenescence

## Typical scenario

In [2]:
def run_simulation(nb_plants, nb_phyelts, nb_lesions):
    # Generate required canopy
    g, domain_area = adel_sample(nb_sect=nb_phyelts, d=1000, p=nb_plants)
    set_properties(g, label = 'LeafElement', 
                   area=5., green_area=5., healthy_area=5., senesced_area=0., position_senescence=1.,
                   temp = 22., wetness = True, relative_humidity = 90, rain_intensity = 0., rain_duration = 0.)
    
    # Initiate all models used for simulation
    septoria = plugin_septoria('septoria_age_physio')
    Lesion = septoria.lesion()
    growth_controler = NoPriorityGrowthControl()
    infection_controler = BiotrophDUPositionModel()
    sen_model = WheatSeptoriaPositionedSenescence(g, label='LeafElement')
    emitter = BenchSeptoriaRainEmission(domain_area=domain_area)
    transporter = Septo3DSplash()
    washor = RapillyWashing()
    
    # Initiate lesions on each phyto-element
    lesions = generate_stock_lesions(nb_lesions=nb_lesions, disease=septoria)
    leaves = get_leaves(g, label = 'LeafElement')
    for leaf in leaves:
        g.node(leaf).lesions = generate_stock_lesions(nb_lesions=nb_lesions, disease=septoria, position=[rd.random(), rd.random()])
    
    # Simulation
    nb_steps=1000
    for i in range(nb_steps):
        sen_model.find_senescent_lesions(g, label = 'LeafElement')
        update_healthy_area(g, label = 'LeafElement')
        infect(g, 1, infection_controler, label='LeafElement')
        update(g, 1, growth_controler, sen_model, label='LeafElement')   
        if i>0 and i%50==0:
            set_properties(g,label = 'LeafElement', rain_intensity = 1., rain_duration = 1.)           
            g,nb = disperse(g, emitter, transporter, "septoria", label='LeafElement')
            wash(g, washor, 1., label='LeafElement')
        else:
            set_properties(g,label = 'LeafElement', rain_intensity = 0., rain_duration = 0.)
            
    return g

## Benchmark for various numbers of plants, phyto-elements by leaf, initial number of lesions by leaf

In [1]:
from openalea.core.path import path

results = {}
def save():
    df = pd.DataFrame(results, columns=['plant', 'phytomer', 'lesion', 'time'])
    df.to_csv('bench.txt', index=False)

def load():
    global results
    df = pd.DataFrame.from_csv('bench.txt',sep=',',index_col=None)
    rec = df.to_records()
    results['plant']=rec.plant.tolist()
    results['phytomer']=rec.phytomer.tolist()
    results['lesion']=rec.lesion.tolist()
    results['time']=rec.time.tolist()

def init():
    global results
    results['plant']=[]
    results['phytomer']=[]
    results['lesion']=[]
    results['time']=[]
    df = pd.DataFrame.from_dict(results)

def merge(results):
    df1 = pd.DataFrame.from_csv('bench.txt',sep=',',index_col=None)
    df2 = pd.DataFrame(results, columns=['plant', 'phytomer', 'lesion', 'time'])
    rec1 = df1.to_records()
    rec2 = df2.to_records()
    results = {}
    results['plant']=rec1.plant.tolist()+rec2.plant.tolist()
    results['phytomer']=rec1.phytomer.tolist()+rec2.phytomer.tolist()
    results['lesion']=rec1.lesion.tolist()+rec2.lesion.tolist()
    results['time']=rec1.time.tolist()+rec2.time.tolist()
    df = pd.DataFrame(results, columns=['plant', 'phytomer', 'lesion', 'time'])
    df.to_csv('bench.txt', index=False)

mresults = {}
def msave():
    df = pd.DataFrame(mresults, columns=['plant', 'phytomer', 'lesion', 'memory'])
    df.to_csv('bench_mem.txt', index=False)

def mload():
    global mresults
    df = pd.DataFrame.from_csv('bench_mem.txt',sep=',',index_col=None)
    rec = df.to_records()
    mresults['plant']=rec.plant.tolist()
    mresults['phytomer']=rec.phytomer.tolist()
    mresults['lesion']=rec.lesion.tolist()
    mresults['memory']=rec.memory.tolist()

def minit():
    global mresults
    mresults['plant']=[]
    mresults['phytomer']=[]
    mresults['lesion']=[]
    mresults['memory']=[]
    df = pd.DataFrame.from_dict(mresults)

def mmerge(results):
    df1 = pd.DataFrame.from_csv('bench_mem.txt',sep=',',index_col=None)
    df2 = pd.DataFrame(results, columns=['plant', 'phytomer', 'lesion', 'memory'])
    rec1 = df1.to_records()
    rec2 = df2.to_records()
    results = {}
    results['plant']=rec1.plant.tolist()+rec2.plant.tolist()
    results['phytomer']=rec1.phytomer.tolist()+rec2.phytomer.tolist()
    results['lesion']=rec1.lesion.tolist()+rec2.lesion.tolist()
    results['memory']=rec1.memory.tolist()+rec2.memory.tolist()
    df = pd.DataFrame(results, columns=['plant', 'phytomer', 'lesion', 'memory'])
    df.to_csv('bench_mem.txt', index=False)


In [4]:
# time benchmark
from datetime import datetime as dt

repeat = 1
def measure(plant=1, phy=1, les=1):
    start = dt.now()
    g = run_simulation(plant, phy, les)
    timelaps = (dt.now()-start).total_seconds()
    return timelaps, g

def bench(plant, phy, les):
    times, gs=zip(*[measure(plant, phy, les) for i in range(repeat)])
    _time =min(times)
    print _time
    results['plant'].append(plant)
    results['phytomer'].append(phy)
    results['lesion'].append(les)
    results['time'].append(_time)
    return gs[0]

def mbench(plant, phy):
    gs, ds=zip(*[adel_sample(nb_sect=phy, d=1000, p=plant) for i in range(repeat)])

    results['plant'].append(plant)
    results['phytomer'].append(phy)
    results['lesion'].append(les)
    results['time'].append(_time)
    return gs[0]